<a href="https://colab.research.google.com/github/poojashreeNS/ANN_Techniques/blob/main/Assignment-5/Assignment_5a_Avalanche_CLEAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As part of this assignment, I have implemented end-to-end continual learning framework for image classification on CLEAR continual learning benchmark using Avalanche library.

Avalanche is an end-to-end Continual Learning library based on Pytorch, born within ContinualAI with the unique goal of providing a shared and collaborative open-source (MIT licensed) codebase for fast prototyping, training and reproducible evaluation of continual learning algorithms.

**Install Avalanch library**

In [ ]:
!pip install avalanche-lib==0.2.1

**Importing the required libraries**

In [ ]:
import os
import sys
import json
from pathlib import Path

import numpy as np
import torch
import torchvision

from avalanche.evaluation.metrics import (
    forgetting_metrics,
    accuracy_metrics,
    loss_metrics,
    timing_metrics,
    cpu_usage_metrics,
    confusion_matrix_metrics,
    disk_usage_metrics,
)
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.training.supervised import Naive
from avalanche.benchmarks.classic.clear import CLEAR, CLEARMetric

**Setting up CLEAR Dataset**

In [ ]:
DATASET_NAME = "clear10"
NUM_CLASSES = {"clear10": 11}
CLEAR_FEATURE_TYPE = "moco_b0"  # MoCo V2 pretrained on bucket 0
EVALUATION_PROTOCOL = "streaming"  # trainset = testset per timestamp

**Initializing paths for saving the datasets/models/results/logfiles**

In [ ]:
ROOT = Path("..")
DATA_ROOT = ROOT / DATASET_NAME
MODEL_ROOT = ROOT / "models"
DATA_ROOT.mkdir(parents=True, exist_ok=True)
MODEL_ROOT.mkdir(parents=True, exist_ok=True)

**Define hyperparameters/model/scheduler/augmentation**

In [ ]:
HPARAM = {
    "batch_size": 512,
    "num_epoch": 10,
    "step_scheduler_decay": 60,
    "scheduler_step": 0.1,
    "start_lr": 1,
    "weight_decay": 0,
    "momentum": 0.9,
}

**Setting parameters for the resnet50 model. Feature size is 2048 for resnet50**

In [ ]:
# feature size is 2048 for resnet50
model = torch.nn.Linear(2048, NUM_CLASSES[DATASET_NAME])

**Defining a scheduler**

In [ ]:
def make_scheduler(optimizer, step_size, gamma=0.1):
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=step_size, gamma=gamma
    )
    return scheduler

**Setting up logging and evaluation metrics**

In [ ]:
# log to Tensorboard
tb_logger = TensorboardLogger(ROOT)

# log to text file
text_logger = TextLogger(open(ROOT / "log.txt", "w+"))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(
        num_classes=NUM_CLASSES[DATASET_NAME], save_image=False, stream=True
    ),
    disk_usage_metrics(
        minibatch=True, epoch=True, experience=True, stream=True
    ),
    loggers=[interactive_logger, text_logger, tb_logger],
)

**Setting up evaluation protocal and downloading the CLEAR dataset - CL Benchmark Creation**

In [ ]:
if EVALUATION_PROTOCOL == "streaming":
    seed = None
else:
    seed = 0

scenario = CLEAR(
    evaluation_protocol=EVALUATION_PROTOCOL,
    feature_type=CLEAR_FEATURE_TYPE,
    seed=seed,
    dataset_root=DATA_ROOT,
)

Files already downloaded and verified
Files already downloaded and verified


**Preparation for CL training and testing**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=HPARAM["start_lr"],
    weight_decay=HPARAM["weight_decay"],
    momentum=HPARAM["momentum"],
)
scheduler = make_scheduler(
    optimizer,
    HPARAM["step_scheduler_decay"],
    HPARAM["scheduler_step"],
)

**Continual learning strategy**

In [ ]:
plugin_list = [LRSchedulerPlugin(scheduler)]
cl_strategy = Naive(
    model,
    optimizer,
    torch.nn.CrossEntropyLoss(),
    train_mb_size=HPARAM["batch_size"],
    train_epochs=HPARAM["num_epoch"],
    eval_mb_size=HPARAM["batch_size"],
    evaluator=eval_plugin,
    device=device,
    plugins=plugin_list,
)

**Training and Testing loop over the stream of experiences - Continual learning**

In [ ]:
print("Starting experiment...")
results = []
print("Current protocol : ", EVALUATION_PROTOCOL)
for index, experience in enumerate(scenario.train_stream):
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    res = cl_strategy.train(experience)
    torch.save(
        model.state_dict(), str(MODEL_ROOT / f"model{str(index).zfill(2)}.pth")
    )
    print("Training completed")
    print(
        "Computing accuracy on the whole test set with"
        f" {EVALUATION_PROTOCOL} evaluation protocol"
    )
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Current protocol :  streaming
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
-- >> Start of training phase << --
100%|██████████| 7/7 [00:00<00:00, 30.01it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55541.3496
	DiskUsage_MB/train_phase/train_stream/Task000 = 55541.3496
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2754
	Loss_MB/train_phase/train_stream/Task000 = 2.0724
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0063
	Time_Epoch/train_phase/train_stream/Task000 = 0.2293
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4676
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.7325
100%|██████████| 7/7 [00:00<00:00, 28.16it/s]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55541.3496
	DiskUsage_MB/train_phase/train_stream/Task000 = 55541.3496
	Loss_Epoch/train_phase/train_stream/Task000 = 1.7378
	Loss_MB/train_phase/train_stream/Task000 = 1.5116
	RunningTime_Epoch/train_pha

**Generating the accuracy matrix**

In [ ]:
num_timestamp = len(results)
accuracy_matrix = np.zeros((num_timestamp, num_timestamp))
for train_idx in range(num_timestamp):
    for test_idx in range(num_timestamp):
        accuracy_matrix[train_idx][test_idx] = results[train_idx][
            f"Top1_Acc_Stream/eval_phase/test_stream/Task00{test_idx}"
        ]
print("Accuracy_matrix : ")
print(accuracy_matrix)
metric = CLEARMetric().get_metrics(accuracy_matrix)
print(metric)

metric_log = open(ROOT / "metric_log.txt", "w+")
metric_log.write(
    f"Protocol: {EVALUATION_PROTOCOL} "
    f"Seed: {seed} "
    f"Feature: {CLEAR_FEATURE_TYPE} \n"
)
json.dump(accuracy_matrix.tolist(), metric_log, indent=6)
json.dump(metric, metric_log, indent=6)
metric_log.close()

Accuracy_matrix : 
[[0.85727273 0.84272727 0.83484848 0.84424242 0.82484848 0.79242424
  0.80121212 0.79484848 0.7930303  0.7769697 ]
 [0.86848485 0.88181818 0.86060606 0.86878788 0.84909091 0.82636364
  0.83181818 0.82424242 0.82545455 0.80484848]
 [0.8769697  0.8869697  0.88333333 0.87909091 0.86545455 0.84575758
  0.84090909 0.83939394 0.83969697 0.81484848]
 [0.88090909 0.89121212 0.88606061 0.89969697 0.87151515 0.85121212
  0.85212121 0.84757576 0.84909091 0.82727273]
 [0.88787879 0.89484848 0.89151515 0.89848485 0.89666667 0.86363636
  0.86909091 0.86181818 0.86787879 0.84181818]
 [0.89393939 0.90151515 0.89424242 0.89939394 0.89606061 0.89212121
  0.87212121 0.86939394 0.88060606 0.84393939]
 [0.89757576 0.90333333 0.90090909 0.90242424 0.9        0.88636364
  0.89121212 0.8769697  0.88272727 0.85030303]
 [0.89666667 0.90272727 0.90242424 0.9069697  0.90121212 0.89272727
  0.89212121 0.8930303  0.8869697  0.85575758]
 [0.89909091 0.90393939 0.90272727 0.90909091 0.90181818 0.89